In [56]:
import sys
import requests
import json
from datetime import datetime
from dateutil.relativedelta import relativedelta
import pandas as pd
import matplotlib as plt
import matplotlib.pyplot as pyplot
import os

# Electricity production prediction - premilinary: https://data.fingrid.fi/en/datasets/242

api_key = '3010957519b04fdc94439b784a78d4fe'

ts=datetime.now()

ts1=ts+relativedelta(hours=-24)
ts2=ts+relativedelta(hours=24)

# Creating start and end time variables
dt_str1 = ts1.strftime("%Y-%m-%dT%H:%M:%S.000Z")
dt_str2 = ts2.strftime("%Y-%m-%dT%H:%M:%S.000Z")

url='https://data.fingrid.fi/api/datasets/242/data'

headers ={'x-api-key': api_key}

params={'startTime': dt_str1, 'endTime': dt_str2, 'pageSize': 2000}

response = requests.get(url, headers=headers,params=params)
x = response.json()
epr_pred_df = pd.DataFrame(x['data'])
print(epr_pred_df)

     datasetId                 startTime                   endTime     value
0          242  2024-10-09T20:45:00.000Z  2024-10-09T21:00:00.000Z  10886.90
1          242  2024-10-09T20:30:00.000Z  2024-10-09T20:45:00.000Z  10894.30
2          242  2024-10-09T20:15:00.000Z  2024-10-09T20:30:00.000Z  10901.90
3          242  2024-10-09T20:00:00.000Z  2024-10-09T20:15:00.000Z  10906.00
4          242  2024-10-09T19:45:00.000Z  2024-10-09T20:00:00.000Z  11084.50
..         ...                       ...                       ...       ...
124        242  2024-10-08T13:45:00.000Z  2024-10-08T14:00:00.000Z   9746.83
125        242  2024-10-08T13:30:00.000Z  2024-10-08T13:45:00.000Z   9723.72
126        242  2024-10-08T13:15:00.000Z  2024-10-08T13:30:00.000Z   9700.34
127        242  2024-10-08T13:00:00.000Z  2024-10-08T13:15:00.000Z   9677.64
128        242  2024-10-08T12:45:00.000Z  2024-10-08T13:00:00.000Z   9485.86

[129 rows x 4 columns]


In [57]:
# Let's drop the first column
epr_pred_df = epr_pred_df.drop(['datasetId'], axis=1)

# Changing the names of the columns
epr_pred_columns = ['Aika', 'Tuotanto (MW)']

if epr_pred_df.columns.tolist() != epr_pred_columns:
    epr_pred_df.columns = ['Ylimääräinen', 'Aika', 'Tuotanto (MW)']

epr_pred_df = epr_pred_df[epr_pred_columns]

# Changing MW to kW
epr_pred_df['Tuotanto (MW)'] = epr_pred_df['Tuotanto (MW)'] * 1000

epr_pred_df.rename(columns={'Tuotanto (MW)': 'Tuotanto (kW)'}, inplace=True)

# Replacing unnecessary letters and numbers
epr_pred_df['Aika'] = epr_pred_df['Aika'].str.replace('T', ' ').str.replace('Z', '').str.replace('.000', '')

print(epr_pred_df.head())

                  Aika  Tuotanto (kW)
0  2024-10-09 21:00:00     10886900.0
1  2024-10-09 20:45:00     10894300.0
2  2024-10-09 20:30:00     10901900.0
3  2024-10-09 20:15:00     10906000.0
4  2024-10-09 20:00:00     11084500.0


In [58]:
# Reversing the values
epr_pred_df['Aika'] = epr_pred_df['Aika'].values[::-1]
epr_pred_df['Tuotanto (kW)'] = epr_pred_df['Tuotanto (kW)'].values[::-1]

print(epr_pred_df)

                    Aika  Tuotanto (kW)
0    2024-10-08 13:00:00      9485860.0
1    2024-10-08 13:15:00      9677640.0
2    2024-10-08 13:30:00      9700340.0
3    2024-10-08 13:45:00      9723720.0
4    2024-10-08 14:00:00      9746830.0
..                   ...            ...
124  2024-10-09 20:00:00     11084500.0
125  2024-10-09 20:15:00     10906000.0
126  2024-10-09 20:30:00     10901900.0
127  2024-10-09 20:45:00     10894300.0
128  2024-10-09 21:00:00     10886900.0

[129 rows x 2 columns]


In [60]:

# Making sure that all time values are datetime objects
epr_pred_df['Aika'] = pd.to_datetime(epr_pred_df['Aika'])

# Taking the hourly mean of electricity production
epr_pred_df.set_index('Aika', inplace=True)
epr_pred_df = epr_pred_df.resample('h').mean().reset_index()

# Restoring the columns
epr_pred_df = epr_pred_df[['Aika', 'Tuotanto (kW)']]

# Removing the duplicates
epr_pred_df = epr_pred_df.loc[~epr_pred_df['Aika'].duplicated(keep='first')]

print(epr_pred_df.head())

                 Aika  Tuotanto (kW)
0 2024-10-08 13:00:00      9646890.0
1 2024-10-08 14:00:00      9942670.0
2 2024-10-08 15:00:00     10193275.0
3 2024-10-08 16:00:00     10416925.0
4 2024-10-08 17:00:00     10514800.0
